# 0 環境準備

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers fugashi ipadic
!pip install demoji
!pip install neologdn

In [ ]:
!wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
!tar -zxvf ldcc-20140209.tar.gz

# 1 データの準備 (jsonファイルの作成)

In [4]:
import glob
import os
import json
import re
import demoji
import neologdn
import string

In [5]:
path = "/content/text"
text_dir = os.listdir(path)
category_list = [f for f in text_dir if os.path.isdir(os.path.join(path, f))]
print(category_list)

['smax', 'livedoor-homme', 'movie-enter', 'peachy', 'sports-watch', 'dokujo-tsushin', 'topic-news', 'kaden-channel', 'it-life-hack']


In [6]:
id_category_list = []
for index, category in enumerate(category_list):
    category_dict = {"id": index, "category": category}
    id_category_list.append(category_dict)
print(id_category_list)

[{'id': 0, 'category': 'smax'}, {'id': 1, 'category': 'livedoor-homme'}, {'id': 2, 'category': 'movie-enter'}, {'id': 3, 'category': 'peachy'}, {'id': 4, 'category': 'sports-watch'}, {'id': 5, 'category': 'dokujo-tsushin'}, {'id': 6, 'category': 'topic-news'}, {'id': 7, 'category': 'kaden-channel'}, {'id': 8, 'category': 'it-life-hack'}]


In [7]:
annotations_list = []
for item in id_category_list:
    file_list = glob.glob(f'text/{item["category"]}/{item["category"]}*.txt')
    for file in file_list:
        annotation_dict = {"file_name": os.path.basename(file), "label": item["id"], "category_name": item["category"]}
        annotations_list.append(annotation_dict)
print(annotations_list[:4])

[{'file_name': 'smax-6732022.txt', 'label': 0, 'category_name': 'smax'}, {'file_name': 'smax-6842882.txt', 'label': 0, 'category_name': 'smax'}, {'file_name': 'smax-6605183.txt', 'label': 0, 'category_name': 'smax'}, {'file_name': 'smax-6805310.txt', 'label': 0, 'category_name': 'smax'}]


In [8]:
json_dict = {"category": id_category_list, "annotations": annotations_list}

In [9]:
json_save_path = "/content/text/dataset.json"
with open(json_save_path, mode="wt", encoding="utf-8") as f:
    json.dump(json_dict, f, indent=4)

# 2 Dataloader作成

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer
torch.manual_seed(0)

In [11]:
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [12]:
class LivedoorDataset(Dataset):
    def __init__(self, tokenizer, text_dir=None):
        self.text_dir = text_dir
        self._load_json()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.annotations_list)

    def __getitem__(self, idx):
        text = self._get_text(self.annotations_list[idx]["category_name"], 
                              self.annotations_list[idx]["file_name"])
        encoding = self.tokenizer(text, return_tensors="pt", max_length=512, padding="max_length", truncation=True)
        encoding = {key: torch.squeeze(value) for key, value in encoding.items()}
        encoding["labels"] = self.annotations_list[idx]["label"]

        return encoding
    
    def _load_json(self):
        with open(os.path.join(self.text_dir, 'dataset.json')) as f:
            self.text_json = json.load(f)
        self.annotations_list = self.text_json["annotations"]
        
    def _get_text(self, category_name, file_name):
        file_path = os.path.join(self.text_dir, category_name, file_name)
        lines = open(file_path).read().splitlines()
        text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す。
        text_preprocessed = self._text_preprocess(text)
        return text_preprocessed
        
    def _text_preprocess(self, text):
        # タブの消去
        text = text.translate(str.maketrans({'\n': '', '\t': '', '\r': '', '\u3000': ''}))

        # URLの消去
        text = re.sub(r'http?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
        text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)

        # 絵文字の消去
        text = demoji.replace(string=text, repl='')

        # 文字の正規化
        text = neologdn.normalize(text)

        # 数字をすべて0に
        text = re.sub(r'\d+', '0', text)

        # 大文字を小文字に
        text = text.lower()

        # 【関連記事, 関連サイト, 関連リンク】以降の消去
        target_list = ['関連記事', '関連サイト', '関連リンク']
        for target in target_list:
          idx = text.find(target)
          text = text[:(idx-1)]

        return text

In [13]:
dataset = LivedoorDataset(tokenizer, "/content/text")
print(len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [7000, len(dataset)-7000])
train_dataloader = DataLoader(train_dataset, batch_size=8)
val_dataloader = DataLoader(val_dataset, batch_size=8)
print(len(train_dataset), len(val_dataset))

7367
7000 367


# 3 モデルの学習

In [14]:
from transformers import BertForSequenceClassification
device = "cuda"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=9).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [15]:
import numpy as np
EPOCH = 10
for epoch in range(EPOCH):
  model.train()
  train_loss = 0
  for i, batch in enumerate(train_dataloader):
    batch = {key: value.to(device) for key, value in batch.items()}
    output = model(**batch)
    loss = output.loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss

  model.eval()
  labels_list, outputs_list = [], []
  for i, batch in enumerate(val_dataloader):
    batch = {key: value.to(device) for key, value in batch.items()}
    labels_list = np.concatenate([labels_list, batch["labels"].cpu().detach().numpy()])
    output = model(**batch)
    output = output.logits.argmax(axis=1).cpu().detach().numpy()
    outputs_list = np.concatenate([outputs_list, output])
  
  accuracy = sum(outputs_list == labels_list) / len(outputs_list) * 100 
  print(f"epoch: {epoch + 1}, train_loss: {round(train_loss.item(), 1)}, accuracy: {round(accuracy, 1)}% {sum(outputs_list == labels_list)}/{len(outputs_list)}")

epoch: 1, train_loss: 568.9, accuracy: 92.1% 338/367
epoch: 2, train_loss: 165.6, accuracy: 94.3% 346/367
epoch: 3, train_loss: 73.4, accuracy: 96.7% 355/367
epoch: 4, train_loss: 43.3, accuracy: 96.2% 353/367
epoch: 5, train_loss: 22.7, accuracy: 97.0% 356/367
epoch: 6, train_loss: 22.6, accuracy: 95.9% 352/367
epoch: 7, train_loss: 12.5, accuracy: 95.9% 352/367
epoch: 8, train_loss: 7.5, accuracy: 95.4% 350/367
epoch: 9, train_loss: 14.2, accuracy: 94.8% 348/367
epoch: 10, train_loss: 7.3, accuracy: 95.9% 352/367


In [16]:
torch.save(model, "/content/drive/MyDrive/youtube/NLP/model.pth")

# Appendix (データについて)

In [17]:
import textwrap

In [18]:
file_path = "text/movie-enter/movie-enter-6617244.txt"
txt_file = open(file_path).read()
print(txt_file)

http://news.livedoor.com/article/detail/6617244/
2012-06-04T08:00:00+0900
ノオミ・ラパスかシャーリーズ・セロンか、『プロメテウス』のワールドプレミアでドレス対決
　リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間5月31日に、ロンドンのレスター・スクウェアにて開催された。

　エリザベス女王陛下即位60周年記念に沸き、至る所に国旗が飾られているロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、会場には約1,500名のファンが集結した。

　『プロメテウス』は、人類史上最大の謎“人類の起源”を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱”が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことを——。

　出演は、『ミレニアム ドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリウッドを代表するオスカー女優シャーリーズ・セロン、『SHAME-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『X-MEN:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源”をビジュアル化した。 

　主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包まれた。ノオミ・ラパスは胸元の空いた黒いドレス、シャーリーズ・セロンは金色の髪によく似合う青のドレスで集まった人々を魅了。

In [19]:
lines = txt_file.splitlines()
print(lines)

['http://news.livedoor.com/article/detail/6617244/', '2012-06-04T08:00:00+0900', 'ノオミ・ラパスかシャーリーズ・セロンか、『プロメテウス』のワールドプレミアでドレス対決', '\u3000リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間5月31日に、ロンドンのレスター・スクウェアにて開催された。', '', '\u3000エリザベス女王陛下即位60周年記念に沸き、至る所に国旗が飾られているロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、会場には約1,500名のファンが集結した。', '', '\u3000『プロメテウス』は、人類史上最大の謎“人類の起源”を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱”が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことを——。', '', '\u3000出演は、『ミレニアム ドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリウッドを代表するオスカー女優シャーリーズ・セロン、『SHAME-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『X-MEN:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源”をビジュアル化した。 ', '', '\u3000主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包ま

In [20]:
text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す。
print(text)

　リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間5月31日に、ロンドンのレスター・スクウェアにて開催された。

　エリザベス女王陛下即位60周年記念に沸き、至る所に国旗が飾られているロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、会場には約1,500名のファンが集結した。

　『プロメテウス』は、人類史上最大の謎“人類の起源”を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱”が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことを——。

　出演は、『ミレニアム ドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリウッドを代表するオスカー女優シャーリーズ・セロン、『SHAME-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『X-MEN:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源”をビジュアル化した。 

　主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包まれた。ノオミ・ラパスは胸元の空いた黒いドレス、シャーリーズ・セロンは金色の髪によく似合う青のドレスで集まった人々を魅了。写真撮影と数台のカメラの取材に応えた後、ファンサービスに終始していた。

　映画『プロメテウス』は、8月24日（金）全国ロードショー＜3D/2D同時上映＞

・映画『プロメテウス』オフィシャルサイト

■関連記事
・人類誕生の謎が明かさ

In [21]:
def text_preprocess(text):
    # タブの消去
    text = text.translate(str.maketrans({'\n': '', '\t': '', '\r': '', '\u3000': ''}))

    # URLの消去
    text = re.sub(r'http?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)

    # 絵文字の消去
    text = demoji.replace(string=text, repl='')

    # 文字の正規化
    text = neologdn.normalize(text)

    # 数字をすべて0に
    text = re.sub(r'\d+', '0', text)

    # 大文字を小文字に
    text = text.lower()

    # 【関連記事, 関連サイト, 関連リンク】以降の消去
    target_list = ['関連記事', '関連サイト', '関連リンク']
    for target in target_list:
      idx = text.find(target)
      text = text[:(idx-1)]

    return text

In [22]:
preprocessed_text = text_preprocess(text)
print(preprocessed_text)

リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間0月0日に、ロンドンのレスター・スクウェアにて開催された。エリザベス女王陛下即位0周年記念に沸き、至る所に国旗が飾られているロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、会場には約0,0名のファンが集結した。『プロメテウス』は、人類史上最大の謎“人類の起源"を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱"が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことをー。出演は、『ミレニアムドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリウッドを代表するオスカー女優シャーリーズ・セロン、『shame-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『x-men:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源"をビジュアル化した。主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包まれた。ノオミ・ラパスは胸元の空いた黒いドレス、シャーリーズ・セロンは金色の髪によく似合う青のドレスで集まった人々を魅了。写真撮影と数台のカメラの取材に応えた後、ファンサービスに終始していた。映画『プロメテウス』は、0月0日(金)全国ロードショー<0d/0d同時上映>・映画『プロメテウス』オフィシャ


In [23]:
s_wrap_list = textwrap.wrap(''.join(preprocessed_text), 100)
print('\n'.join(s_wrap_list), "\n")

リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間0月0日に、ロンドンのレスター・スクウェアにて開催された。エリザベス女王陛下即位0周年記念に沸き、至る所に国旗が飾られている
ロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、
会場には約0,0名のファンが集結した。『プロメテウス』は、人類史上最大の謎“人類の起源"を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの
、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱"が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上
のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことをー。出演は、『ミレニアムドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリ
ウッドを代表するオスカー女優シャーリーズ・セロン、『shame-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『x-men:ファースト・ジェネレーション』を崇高なドラマにまで昇華
させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心
や空想をはるかに超越した“人類の起源"をビジュアル化した。主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に
包まれた。ノオミ・ラパスは胸元の空いた黒いドレス、シャーリーズ・セロンは金色の髪によく似合う青のドレスで集まった人々を魅了。写真撮影と数台のカメラの取材に応えた後、ファンサービスに終始していた。映画『
プロメテウス』は、0月0日(金)全国ロードショー<0d/0d同時上映>・映画『プロメテウス』オフィシャ 

